# Множественное тестирование гипотез

In [ ]:
import warnings
warnings.filterwarnings("ignore")

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !wget -q -O requirements.txt https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/seminars/sem4/requirements.txt
    !pip install -q -r requirements.txt

## Библиотеки

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

import scipy.stats as st
import seaborn as sns

# критерий стюдента для независимых выборок
from  statsmodels.stats.weightstats import ttest_ind

# пакет для работы с множественным тестированием
from statsmodels.stats.multitest import multipletests

# пакет для проверки множественных гипотез
from statsmodels.stats.multitest import multipletests

## Warmup

### Модельный эксперимент

#### Постановка
**Данные:**

$n=20, \quad m=200, \quad m_0=150$,

$\mathbf{X}_i \sim \mathcal{N}(\mathbf{0}, \mathbf{I}), \quad i=1,\dots,m_0;$

$\mathbf{X}_i \sim \mathcal{N}(\mathbf{1}, \mathbf{I}), \quad i=m_0+1,\dots,m;$

$\mathbf{X}_i \in \mathbb{R}^n.$

**Гипотезы:**

$H^i_0: \mathsf{E}\mathbf{X}_i = 0$;

$H^i_1: \mathsf{E}\mathbf{X}_i \neq 0$;


**Критерий для проверки:**
* одновыборочный t-Критерия Стьюдента.

#### Генерация данных

In [ ]:
rs = np.random.RandomState(42)

alpha = 0.05
m = 200
m0 = 150
n = 20

X = rs.randn(m,n)
X[m0:]+=1

#### Гистограммы

In [ ]:
_ = plt.hist(X[:m0].flatten(), color='r', density=True)
_ = plt.hist(X[m0:].flatten(), color='g', alpha=0.5, density=True)

#### Эксперимент

В качестве статистики использовать функцию ```st.ttest_1samp``` из пакета ```scipy.stats```.

##### Без поправок

In [ ]:
# zero mean
rejected = 0
for x in X[:m0]:
    if st.ttest_1samp(x, 0, )[1] < 0.05:
         rejected += 1

print ('True: H0; H1 non-accepted:', m0 - rejected)
print ('True: H0; H0 rejected:', rejected)

# non-zero mean
rejected = 0
for x in X[m0:]:
    if st.ttest_1samp(x, 0)[1] < 0.05:
         rejected += 1

print ('False: H1; H1 non-accepted:', m-m0 - rejected)
print ('False: H1; H0 rejected:', rejected)

##### Бонферони (из пакета)

In [ ]:
# zero mean
p_values = []
for x in X:
    ### Ваш код тут ###
    p_values.append(st.ttest_1samp(x, 0, )[1])

rejections = multipletests(p_values, method='bonferroni')[0]

print ('True: H0; H1 non-accepted:', m0 - rejections[:m0].sum())
print ('True: H0; H1 rejected:',  rejections[:m0].sum())

# non-zero mean
rejections = multipletests(p_values, method='bonferroni')[0]

print ('True: H1;  H1 non-accepted:', m-m0 - rejections[m0:].sum())
print ('True: H1;  H1 rejected:',  rejections[m0:].sum())



##### Бонферони (руками)

In [ ]:
# zero mean
rejected = 0
for x in X[:m0]:
    if st.ttest_1samp(x, 0, )[1] < 0.05/200:
         rejected += 1

print ('True: H0; H1 non-accepted:', m0 - rejected)
print ('True: H0; H0 rejected:', rejected)

# non-zero mean
rejected = 0
for x in X[m0:]:
    if st.ttest_1samp(x, 0, )[1] < 0.05/200:
         rejected += 1

print ('True: H1; H1 non-accepted:', m-m0 - rejected)
print ('True: H1; H0 rejected:', rejected)

##### График зависимости мощности для разных поправок

In [ ]:
# no correction
plt.plot([0, m - 1], [alpha] * 2, label='No correction')

# bonferroni
bonferroni = alpha / m
plt.plot([0, m - 1], [bonferroni] * 2, label='Bonferroni')

# holm
holm = alpha / np.arange(m, 0, -1)
plt.plot(holm, label='Holm')

# sidak
sidak = 1 - np.power((1. - alpha),  1. / np.arange(m, 0, -1))
plt.plot(sidak, label='Sidak', ls='--')

# benjamini-hochberg
ecdffactor = np.arange(1, m + 1)/float(m)
bh = alpha * ecdffactor
plt.plot(range(m), bh, label='Benjamini-Hochberg')

# benjamini-yekutieli 
cm = np.sum(1. / np.arange(1, m+1)) 
ecdffactor = ecdffactor / cm
by = alpha * ecdffactor
plt.plot(range(m), by, label='Benjamini-Yekutieli')

plt.xlabel('i')
plt.ylabel('$a_i$')
plt.legend(loc='best')

plt.show()

##### График модифицированых уровней значимости

In [ ]:
def plot_diff_alpha(p_values, loc_plt=None):
    '''
    Функция для отрисовки модифицированых, отсортированых 
    уровней значимости для модельной выборки.
    
    param: p_values --- список уровней значимости
    param: loc_plt --- объект matplotlib.pylab где производится отрисовка
    
    return: loc_plt --- объект matplotlib.pylab после отрисовки
    '''
    if loc_plt is None:
        return None
    argsorted_p = np.argsort(p_values)
    false = []
    true = []
    for i, id in enumerate(argsorted_p):   
        if id>m0:
            false.append((i, p_values[id]))
        else:
            true.append((i, p_values[id]))
    
    loc_plt.scatter(*zip(*true), label='True hypothesis', c='b')
    loc_plt.scatter(*zip(*false), label='False hypothesis', c='r')
    loc_plt.xlabel('sorted i')
    loc_plt.ylabel('$\hat{p}(i)$')
    loc_plt.legend(loc='best')
    
    return loc_plt

##### График модифицированых уровней значимости (без поправок)

In [ ]:
p_values = []
for x in X:
     p_values.append(st.ttest_1samp(x, 0)[1])
        
plot_diff_alpha(p_values, loc_plt=plt)
plt.show()

##### График модифицированых уровней значимости (Bonferroni)

In [ ]:
p_values = []
for x in X:
     p_values.append(st.ttest_1samp(x, 0)[1])
        
p_values = multipletests(p_values, method='bonferroni')[1]

plot_diff_alpha(p_values, loc_plt=plt)
plt.show()

##### График модифицированых уровней значимости (Holm)

In [ ]:
p_values = []
for x in X:
     p_values.append(st.ttest_1samp(x, 0)[1])
        
p_values = multipletests(p_values, method='holm')[1]

plot_diff_alpha(p_values, loc_plt=plt)
plt.show()

##### График модифицированых уровней значимости (Holm-Sidak)

In [ ]:
p_values = []
for x in X:
     p_values.append(st.ttest_1samp(x, 0)[1])
        
p_values = multipletests(p_values, method='holm-sidak')[1]

plot_diff_alpha(p_values, loc_plt=plt)
plt.show()

##### График модифицированых уровней значимости (Benjamini-Hochberg)

In [ ]:
p_values = []
for x in X:
     p_values.append(st.ttest_1samp(x, 0)[1])
        
p_values = multipletests(p_values, method='fdr_bh', )[1]

plot_diff_alpha(p_values, loc_plt=plt)
plt.show()

##### График модифицированых уровней значимости (Benjamini-Yekutieli)

In [ ]:
p_values = []
for x in X:
     p_values.append(st.ttest_1samp(x, 0)[1])
        
p_values = multipletests(p_values, method='fdr_by', )[1]

plot_diff_alpha(p_values, loc_plt=plt)
plt.show()

## Лейкоциты и астма

С использованием ДНК-микрочипов были получены анализ изменения генной экспрессии для двух групп детей: больных астмой и здоровых.

Источник данных: http://www.ncbi.nlm.nih.gov/sites/GDSbrowser?acc=GDS4896

### Рассмотрим сами данные

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/seminars/sem4/data/expression.csv', delimiter=';')

In [ ]:
data.head(5)

In [ ]:
len(data)

In [ ]:
print('\n'.join(list(data.columns)))

### Эксперимент

#### Достигаемые уровни значимости в t-Критерии Стьюдента

In [ ]:
data_np = data.values
def get_healthy(i):
    '''
    Возвращает всех здоровых пациентов для i-го микрочипа
    '''
    return data_np[i, 2:20]

def get_asthma(i):
    '''
    Возвращает всех больных астмой пациентов для i-го микрочипа
    '''
    return data_np[i, 39:56]

def get_pvalue_t_test(i):
    '''
    Возвращает p-value для критерия Стьюдента для независимых выборок
    '''
    ### Ваш код тут ###
    return ttest_ind(get_healthy(i), get_asthma(i), usevar='unequal')[1]
    
p_values = list(map(get_pvalue_t_test, range(data_np.shape[0])))
p_values = np.array(p_values)

#### Модифицированые уровни значимости

In [ ]:
### Ваш код тут ###

### Ваш код тут ###

p_holm = multipletests(p_values, method='holm')[1]

p_bh = multipletests(p_values, method='fdr_bh', )[1]

In [ ]:
p_values

#### Распределения исходных и модифицированных достигаемых уровней значимости

In [ ]:
plt.plot(sorted(p_values), label='no correction')

plt.plot(sorted(p_holm), label='holm')

plt.plot(sorted(p_bh), label='bh')

plt.xlabel('sorted i')
plt.ylabel('$\hat{p}(i)$')
plt.legend(loc='best')
plt.show()

#### Количество генов, экспрессия которых статистически значимо отличается при поправке методами Холма и Бенджамини-Хохберга:

In [ ]:
print('Количество генов для которых значимо отличается: {}\t(no correction)'.format((p_values <= 0.05).sum()))
print('Количество генов для которых значимо отличается: {}\t(holm)'.format((p_holm <= 0.05).sum()))
print('Количество генов для которых значимо отличается: {}\t(bh)'.format((p_bh <= 0.05).sum()))

### На микрочипах есть много проб к малоизученным участкам генома, у которых даже нет названия; их можно сразу удалить из рассмотрения (Эксперимент 2)

In [ ]:
reduced_data = data[data['Gene title'].isnull() == False]
reduced_data.shape, data.shape

#### Рассмотрим как изменились уровни значимости

In [ ]:
data_np = reduced_data.values

### Ваш код тут ###
p_values = list(map(get_pvalue_t_test, range(data_np.shape[0])))
p_values = np.array(p_values)

In [ ]:
### Ваш код тут ###

### Ваш код тут ###

p_holm = multipletests(p_values, method='holm')[1]

p_bh = multipletests(p_values, method='fdr_bh')[1]

In [ ]:
plt.plot(sorted(p_values), label='no correction')

plt.plot(sorted(p_holm), label='holm')

plt.plot(sorted(p_bh), label='bh')

plt.xlabel('sorted i')
plt.ylabel('$\hat{p}(i)$')
plt.legend(loc='best')
plt.show()

In [ ]:
print('Количество генов для которых значимо отличается: {}\t(no correction)'.format((p_values <= 0.05).sum()))
print('Количество генов для которых значимо отличается: {}\t(holm)'.format((p_holm <= 0.05).sum()))
print('Количество генов для которых значимо отличается: {}\t(bh)'.format((p_bh <= 0.05).sum()))

## Сравнение качества классификаторов

Дано: AUC различных версий алгоритма C4.5, посчитанный на 15 датасетах.

**Demsar J. (2006). Statistical Comparisons of Classifiers over Multiple Data Sets. Journal of Machine Learning Research, 7, 1–30.**

### Рассмотрим данные

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/seminars/sem4/data/auc.txt', delimiter='\t')

In [ ]:
data.head(5)

### Эксперимент

Между какими из версий есть значимые различия?

Сравним все значения критерия качества попарно с помощью критерия знаковых рангов Уилкоксона (```st.wilcoxon```):

In [ ]:
p_values= []
for i in range(1, 5):
    for j in range(i+1, 5):
        test_result = st.wilcoxon(data.iloc[:, i].values, data.iloc[:, j].values) ### Ваш код тут ###
        p_values += [test_result[1]]
        print(data.columns[i], data.columns[j], p_values[-1])

### Разные поправки

In [ ]:
for k in ['bonferroni', 'holm', 'fdr_bh', 'fdr_by']:
    rejects, p_val, *_ = multipletests(p_values, method=k)

    print(f'{k}\n{str(rejects)}\n{str(p_val)}\n\n')